# Programme de Matching

Regarder si l'établissement est fermé ou pas

## Moteur de recherche TEST

* Insee
  * http://avis-situation-sirene.insee.fr/IdentificationListeSiret.action
* INPI/TC
  * https://data.inpi.fr/
* Infogreffe
  * https://www.infogreffe.fr/

## Preparation fichier

### Variable Ville

- process:
    - creer variables avec numeric seulement
    - recreer ville 2 si test pas NAN pour avoir l'arrondissement
    - virer les differentes informations dans ville via regex
    
### Step 1: Match uniquement les 1 dans INSEE/INPI

Les siren qui ont uniquement un seul établissement a l'INSEE et à l'INPI.

### Step 2: match Ville/CP/CC

### Step 3: Match adresse

## Description Algorithm

## Description Input

## Description Output

In [ ]:
from inpi_insee import siretisation

%load_ext autoreload
%autoreload 2

In [ ]:
param = {
    'insee': r'data\input\insee_2017_7627977.gz'
}
# 4824158 SIREN a trouver!
al_siret = siretisation.siretisation_inpi(param)

### Match 1-1

### Merge ville/code postale/ code commune

Process temporaire -> on sauvegarde les fichiers avec indication des tests:

## First Step:

- Après le merge full
    - Test 1: doublon
        - non: Save-> `test_1['not_duplication']`
        - oui:
            - Test 2: Date equal
                - oui:
                    - Test 2 bis: doublon
                        - non: Save-> `test_2_bis['not_duplication']`
                        - oui: Save-> `test_2_bis['duplication']`
                - non:
                    - Test 3: Date sup
                        - oui:
                            - Test 2 bis: doublon
                                - non: Save-> `test_3_oui_bis['not_duplication']`
                                - oui: Save-> `test_3_oui_bis['duplication']`
                        - non: Save-> `test_3_non`
                        
## Second Step:

On applique 5 tests sur les champs des possibles afin d'avoir une meilleure visibilitée 

- Test 1: address libelle
    - Si mots dans inpi est contenu dans INSEE, True
- Test 1 bis: address complement
    - Si mots dans inpi est contenu dans INSEE, True
- Test 2: Date
    - dateCreationEtablissement >= Date_Début_Activité OR Date_Début_Activité = NaN OR (nombre SIREN a l'INSEE = 1 AND nombre SIREN des variables de matching = 1), True
- Test 3: siege
    - Type = ['SEP', 'SIE'] AND siege = true, True
- Test 4: voie
    - Type voie INPI = Type voie INSEE, True
- Test 5: numero voie
    - Numero voie INPI = Numero voie INSEE, True
    
### Second Steps: not on dupplication: df_no_duplication

- On applique simplement les tests, mais on recupère toute la df car considérée comme matchée

### Second Steps: on dupplication: df_duplication

- On applique  les tests, mais on effectue les tests suivants:

- Si test_join_address = True:
    - Test 1: doublon:
        - Oui: append-> `df_not_duplicate`
        - Non: Pass
        - Exclue les `index` de df_duplication
        - then go next
    - Si test_address_libelle = True:
        - Test 1: doublon:
            - Oui: append-> `df_not_duplicate`
            - Non: Pass
            - Exclue les `index` de df_duplication
            - then go next
    - Si test_address_complement = True:
        - Test 1: doublon:
            - Oui: append-> `df_not_duplicate`
            - Non: Pass
            - Exclue les `index` de df_duplication

On peut sauvegarder le `df_not_duplicate` et le restant en tant que `special_treatment`

Retourne au début, et on réduit les variables de matching.

In [ ]:
#from itertools import compress, product

#def combinations(items):
#    return ( set(compress(items,mask)) for 
#            mask in product(*[[0,1]]*len(items)))

#all_list = ['ncc',
#             'Code_Postal','Code_Commune',
#             'INSEE','digit_inpi']
#test = list(combinations(items = all_list))[1:]
#sort_list = sorted(test[1:], key=lambda k: len(k), reverse=True) 

In [ ]:
list_inpi = ['ncc','Code_Postal','Code_Commune','INSEE','digit_inpi']
list_insee = ['libelleCommuneEtablissement',
            'codePostalEtablissement', 'codeCommuneEtablissement',
            'typeVoieEtablissement','numeroVoieEtablissement']

sort_list = [
 {'ncc', 'Code_Postal', 'Code_Commune', 'INSEE', 'digit_inpi'},
 {'ncc', 'Code_Postal', 'Code_Commune', 'INSEE'},
 {'ncc', 'Code_Postal', 'Code_Commune', 'digit_inpi'},
 {'ncc', 'Code_Postal', 'Code_Commune'},   
 {'ncc', 'Code_Postal'},
 {'ncc'},
 {'Code_Postal'},
 {'Code_Commune'}
]
len(sort_list)

In [ ]:
list_possibilities = []
for i in sort_list:
    left =[]
    right = []
    for j in i:
        left.append(j)
        right.append(list_insee[list_inpi.index(j)])
    left.insert(0,'siren')
    right.insert(0,'siren')
    
    dic_ = {
    'match':{
        'inpi':left,
        'insee':right,
    }
}
    list_possibilities.append(dic_)

In [ ]:
%%time
import json
inpi_col = ['siren',
            'index',
            'Type',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adresse_new_clean_reg',
            'Adress_new',
            'INSEE',
            'Date_Début_Activité',
            'digit_inpi',
            'len_digit_address_inpi',
            'list_digit_inpi'
            ]

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adresse_new_clean_reg': 'object',
    'Adress_new':'object',
    'INSEE': 'object',
    'Date_Début_Activité': 'object',
    'digit_inpi': 'object',
    'len_digit_address_inpi':'object'
}

for key, values in enumerate(list_possibilities):
    df_ets = r'data\input\INPI\inpi_etb_stock_{}.gz'.format(key)

    inpi = al_siret.import_dask(file=df_ets,
                                usecols=inpi_col,
                                dtype=inpi_dtype,
                                parse_dates=False)

    df_no_duplication, df_duplication = al_siret.step_one(
        df_input=inpi,
        left_on=values['match']['inpi'],
        right_on=values['match']['insee']
    )

    # Step 2: No duplication
    pure_match = al_siret.step_two_assess_test(df=df_no_duplication,
                                               var_group=values['match']['inpi'])

    pure_match.to_csv(r'data\output\{}_pure_match.gz'.format(key),
                      compression='gzip', index= False)
    # Step 2: duplication
    df_not_duplicate, sp = al_siret.step_two_duplication(df_duplication,
                                                        var_group = values['match']['inpi'])
    
    (df_not_duplicate
        .to_csv(r'data\output\{}_not_duplicate.gz'.format(key),
                compression='gzip', index= False))

    (sp.to_csv(r'data\input\INPI\special_treatment\{}_special_treatment.gz'.format(
        key),compression='gzip', index= False))

    # Input -> Save for the next loop 
    inpi.loc[
        (~inpi['index'].isin(pure_match['index'].unique()))
        & (~inpi['index'].isin(df_not_duplicate['index'].unique()))
        & (~inpi['index'].isin(sp['index'].unique()))
    ].compute().to_csv(r'data\input\INPI\inpi_etb_stock_{}.gz'.format(key+1),
                       compression='gzip', index= False)

    #### Creation LOG
    if key ==0:
        total_to_siret_intial = inpi.compute().shape[0]
        total_siren_initial = inpi.compute()['siren'].nunique()
    
    ### Total rows in df inpi to match
    total_to_siret_current = inpi.compute().shape[0]
    total_siren_current = inpi.compute()['siren'].nunique() # unique siren 
    
    ### DF with no duplication after merge INSEE
    total_rows_no_dup = df_no_duplication["index"].nunique()
    total_rows_no_dup_unique_siren = df_no_duplication['siren'].nunique()
    
    ### DF with duplication after merge INSEE
    total_rows_dup = df_duplication["index"].nunique() # total duplication
    total_rows_dup_unique_siren = df_duplication["siren"].nunique()
    
    total_rows_dup_matched = df_not_duplicate["index"].nunique() #no duplication
    total_rows_dup_matched_unique_siren = df_not_duplicate["siren"].nunique()
    
    total_rows_dup_not_matched = sp["index"].nunique() # special treatmnent
    total_rows_dup_not_matched_unique_siren = sp["siren"].nunique()
    
    ### compare with initial
    total_match_rows_current = total_rows_no_dup + total_rows_dup_matched
    perc_total_match_rows_initial = total_match_rows_current / \
    total_to_siret_intial
    
    total_match_siren_current = total_rows_no_dup_unique_siren + \
    total_rows_dup_matched_unique_siren
    
    perc_total_match_siren_initial = total_match_siren_current / \
    total_siren_initial 
    
    ### compare with current
    perc_total_match_rows_current = total_match_rows_current / \
    total_to_siret_current

    perc_total_match_siren_current = total_match_siren_current / \
    total_siren_current
    
    
    dic_ = {
        'key':key,
        'total_to_siret_intial':total_to_siret_intial,
        'total_stotal_siren_initialiren': total_siren_initial,
        'total_to_siret_current':total_to_siret_current,
        'total_siren_current': total_siren_current,
        'total_match_rows_current':total_match_rows_current,
        'perc_total_match_rows_initial':perc_total_match_rows_initial,
        'total_match_siren_current':total_match_siren_current,
        'perc_total_match_siren_initial':perc_total_match_siren_initial,
        'perc_total_match_rows_current':perc_total_match_rows_current,
        'perc_total_match_siren_current':perc_total_match_siren_current,
        'df_no_duplication': {
            'nb_index': total_rows_no_dup,
            'unique_siren':total_rows_no_dup_unique_siren
        },
        'df_duplication': {
            'nb_index': total_rows_dup,
            'unique_siren':total_rows_dup_unique_siren,
            'df_not_duplicate_index': {
                'nb_index':total_rows_dup_matched,
               'unique_siren':total_rows_dup_unique_siren
            },
            'df_sp_index': {
                'nb_index':total_rows_dup_not_matched,
               'unique_siren':total_rows_dup_not_matched_unique_siren
            }
        },
        'check': total_to_siret_current -
        total_rows_no_dup +
        total_rows_dup
    }

    with open(r'data\logs\{}_logs.json'.format(key), 'w') as f:
        json.dump(dic_, f)

### temp test

In [ ]:
sp.head().to_excel('test.xlsx')

In [ ]:
inpi_col = ['siren',
            'index',
            'Type',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adresse_new_clean_reg',
            'INSEE',
            'Date_Début_Activité',
            'digit_inpi',
            'len_digit_address_inpi',
            'list_digit_inpi'
            ]

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adresse_new_clean_reg': 'object',
    'INSEE': 'object',
    'Date_Début_Activité': 'object',
    'digit_inpi': 'object',
    'len_digit_address_inpi':'object'
}
df_ets = r'data\input\INPI\inpi_etb_stock_{}.gz'.format(4)

inpi = al_siret.import_dask(file=df_ets,
                                usecols=inpi_col,
                                dtype=inpi_dtype,
                                parse_dates=False)
inpi[inpi['siren'].isin(['752085324'])].compute()

In [ ]:
inpi.compute().shape[0]

In [ ]:
inpi.compute()['siren'].nunique()

# Open Logs

In [ ]:
import glob, os, json
data = []
#os.chdir(r"data\logs\")
for file in glob.glob(r"data\logs\*.json"):
    with open(file) as f:
        for line in f:
            data.append(json.loads(line))

In [ ]:
#from pandas.io.json import json_normalize
import pandas as pd

In [ ]:
logs = pd.json_normalize(data)
logs

In [ ]:
logs[['total_match_rows_current']].sum()

In [ ]:
logs[['perc_total_match_rows_initial']].sum()

In [ ]:
logs[['perc_total_match_siren_initial']].sum()

In [ ]:
logs[['perc_total_match_rows_initial',
      'perc_total_match_siren_initial']].plot.bar(stacked=False)

In [ ]:
logs[['total_match_rows_current']].plot.bar(stacked=False)

# Special Treatment

exemple:

- 752085324
- 342122546: libelle type dans l'adresse complementaire a l'insee

## Recuperation via list_digit_ INPI/INSEE

In [ ]:
import pandas as pd

In [ ]:
for i in range(0,8):
    test = pd.read_csv(
        r'data\input\INPI\special_treatment\{}_special_treatment.gz'.format(i),
                   low_memory=False)
    print((test
 .loc[lambda x:
      (x['list_digit_inpi'] == x['list_digit_insee'])
     ]
     )['siren'].nunique())

In [ ]:
#.assign(special_digit = lambda x:x['libelleVoieEtablissement'].str.findall(r"(\d+)").apply(
#        lambda x:'&'.join([i for i in x])))

## Archive

In [ ]:
inpi_col = ['siren',
            'index',
            'Type',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adresse_new_clean_reg',
           'INSEE',
           'Date_Début_Activité',
           'digit_inpi']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adresse_new_clean_reg': 'object',
    'INSEE':'object',
    'Date_Début_Activité':'object',
    'digit_inpi':'object'
}

df_ets = r'data\input\inpi_etb_stock_{}.gz'.format(0)
inpi = al_siret.import_dask(file=df_ets,
                            usecols=inpi_col,
                            dtype=inpi_dtype,
                            parse_dates = False)
    
df_no_duplication, df_duplication = al_siret.step_one(
    df_input=inpi,
    left_on=list_possibilities[0]['match']['inpi'],
    right_on=list_possibilities[0]['match']['insee']
    )

In [ ]:
df_not_duplicate, sp = al_siret.step_two_duplication(df_duplication,
                                                    var_group=list_possibilities[0]['match']['inpi'])

In [ ]:
sp["index"].nunique()

In [ ]:
dic_

In [ ]:
##### POUR LOG
df_no_duplication.groupby('origin')['origin'].count()
df_duplication.groupby('origin')['origin'].count()
doublou_sur['index'].nunique()
doublou_passur['index'].nunique()
doublon.shape[0] - ( doublou_sur.shape[0] + doublou_passur.shape[0])
pure_match['count_duplicates_final'].value_counts()
pure_match['index'].nunique() + df_not_duplicate['index'].nunique() + sp['index'].nunique()
inpi['index'].compute().nunique()

# Archive

In [ ]:
df_ets = r'data\input\inpi_etb_stock_7552898.gz'
inpi = al_siret.import_dask(file=df_ets,
                       usecols=inpi_col, dtype=inpi_dtype)

df_nomatch = pd.DataFrame()
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                  option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='1_ville_cp_cc',
        to_csv=True)

### Loop

In [ ]:
%%time
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


#df_nomatch = pd.DataFrame()
df_match = pd.DataFrame()
for chunk in [0,1
              ,2,3,4,5,6,7,8,9, 10,11,12
             ]:
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(chunk)
    inpi = al_siret.import_dask(file=csv_file,
                       usecols=inpi_col, dtype=inpi_dtype)
    df_input = inpi
    total_match = pd.DataFrame()
    for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
        df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                      option=i,
                                      regex_go = True)

        df_input = df_input_['unmatch']
        total_match = total_match.append(df_input_['true_match']) 
    #df_nomatch = df_nomatch.append(df_input)
    df_match = df_match.append(total_match)

In [ ]:
df_temp = pd.DataFrame()
for i in range(0,13):
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(i)
    df_temp = df_temp.append(pd.read_csv(csv_file,
                                         dtype= inpi_dtype)
                            )

In [ ]:
299986 / 4979272

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial=df_temp,
        df_inpi_mergeboth=df_match.drop(columns = '_merge'),
        step='3_adress',
        to_csv=True)

### Matching voie

In [ ]:
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

In [ ]:
#temp = al_siret.merge_siren_candidat(df_input=inpi,
#                              regex_go=True, 
#                              matching_voie=True,
#                              option=['ncc', 'libelleCommuneEtablissement'])

In [ ]:
import pandas as pd
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_3_adress_299986.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='4_voie',
        to_csv=True)

### Relax contrainte Regex

Sans numero de voie

In [ ]:
import pandas as pd
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_4_voie_167801.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='5_voie_relax',
        to_csv=True)

Avec la voie

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_5_relax_116575.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=True,
        relax_regex = True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_5_relax_116575.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
check = al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='6_voie_relax',
        to_csv=False)

In [ ]:
check.head()

In [ ]:
check['count_initial_inpi'].value_counts()

### Match etablissement principal ouvert

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_6_voie_relax_112847.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        siege_etat=True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])
    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='7_siege_ouvert',
        to_csv=True)

### Complement d adresse

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_7_siege_ouvert_85312.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        siege_etat=True,
        option=i,
        var_adress_insee = 'complementAdresseEtablissement')

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])
    


In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='8_siege_ouvert_complement',
        to_csv=True)

In [ ]:
1- (80474/4979272)

### Test sans regex mais siege_etat

ie si principale-ouvert dans la ville/CP/CC

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_8_siege_ouvert_complement_80474.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=False,
        matching_voie=False,
        relax_regex = False,
        siege_etat=True,
        option=i,
        var_adress_insee = 'libelleVoieEtablissement')

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='9_siege_ouvert_no_regex',
        to_csv=True)

In [ ]:
round(1 - (68131 / 4979272),3)

# Temp

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\output\match_9_siege_ouvert_no_regex_12343.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
inpi.head()

In [ ]:
inpi['Adresse_new_clean_reg'].head()

## Amelioration matching

In [ ]:
import pandas as pd
insee_dtype = {
    'siren': 'object',
    'siret': 'object',
    'dateCreationEtablissement': 'object',
    "etablissementSiege": "object",
    "etatAdministratifEtablissement": "object",
    'complementAdresseEtablissement': 'object',
    'numeroVoieEtablissement': 'object',
    'indiceRepetitionEtablissement': 'object',
    'typeVoieEtablissement': 'object',
    'libelleVoieEtablissement': 'object',
    'codePostalEtablissement': 'object',
    'libelleCommuneEtablissement': 'object',
    'libelleCommuneEtrangerEtablissement': 'object',
    'distributionSpecialeEtablissement': 'object',
    'codeCommuneEtablissement': 'object',
    'codeCedexEtablissement': 'object',
    'libelleCedexEtablissement': 'object',
    'codePaysEtrangerEtablissement': 'object',
    'libellePaysEtrangerEtablissement': 'object',
    'count_initial_insee': 'int'
}

insee_col = ['siren',
             'siret',
             'dateCreationEtablissement',
             "etablissementSiege",
             "etatAdministratifEtablissement",
             'complementAdresseEtablissement',
             'numeroVoieEtablissement',
             'indiceRepetitionEtablissement',
             'typeVoieEtablissement',
             'libelleVoieEtablissement',
             'codePostalEtablissement',
             'libelleCommuneEtablissement',
             'libelleCommuneEtrangerEtablissement',
             'distributionSpecialeEtablissement',
             'codeCommuneEtablissement',
             'codeCedexEtablissement',
             'libelleCedexEtablissement',
             'codePaysEtrangerEtablissement',
             'libellePaysEtrangerEtablissement',
             'count_initial_insee']
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

insee = al_siret.import_dask(file=r'data\input\insee_2017_7480120.gz',
                             usecols=insee_col, dtype=insee_dtype)


In [ ]:
list(insee)

In [ ]:
#insee['etatAdministratifEtablissement'].value_counts().compute()
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_8_siege_ouvert_complement_80474.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
inpi.head()

In [ ]:
insee[
    (insee['siren'].isin(['515226462']))
#& (insee['libelleVoieEtablissement'].isin(['JACQUARD']))
].compute()

In [ ]:
inpi[inpi['siren'].isin(['515226462'])].compute()

In [ ]:
inpi.loc[inpi['count_initial_inpi'].isin([1])].head(10)

In [ ]:
inpi.loc[inpi['siren'].isin(['322385949'])].compute()

In [ ]:
inpi['Type'].value_counts().compute()

In [ ]:
insee[insee['siren'].isin(['322385949'])].compute()

Verification conservative:

- A l'INSEE, si etatAdministratifEtablissement == A & etablissementSiege == True, alors on fait le sous ensemle
- Verifier avec les dates de création de l'établissement

In [ ]:
insee.loc[
            (insee['etablissementSiege'].isin(['true'])) 
            & (insee['etatAdministratifEtablissement'].isin(['A']))
            ].compute() 

In [ ]:
1 - (112847 / 4979272)

In [ ]:
amelioration possible:
    
- 381980788 -> Zone Artisanale du Creusât ->transformer en ZA